# Task
We want to achieve the following workflow:

Step 1. 
1. Echo a string to msg.txt 
    
2. Output the length of the string as a number
    
Step 2.
1. Duplicate the content in the file from step 1 two times and redirect it to a new file
    
2. Get the number from step 1 and times the number by 2 and output it

##### In this tutorial, we will complete the task using function OP

import necessary packages

In [ ]:
import sys
from pathlib import Path

from dflow import Workflow
from dflow.python import OP, Artifact

For step 1: 

This OP is to write files. In the example, we try to write a file containing message of string format, and output a number of int format.
- input:
    - "msg": the input message
- output:
    - "out_art": file containing the input message
    - "length": length of the input message

In [ ]:
@OP.function(image=f"python:{sys.version_info.major}.{sys.version_info.minor}")
def write_file(msg: str) -> {"out_art": Artifact(Path), "length": int}:
    with open("msg.txt","w") as f:
        f.write(msg)
    
    return {
        "out_art": Path("msg.txt"),
        "length": len(msg),
    }

For Step 2:

This OP is to duplicate the content in the file from in_art, and to multiply the input number by 2.

In [ ]:
@OP.function(image=f"python:{sys.version_info.major}.{sys.version_info.minor}")
def duplicate(in_art: Artifact(Path), in_num: int) -> {"out_art": Artifact(Path), "out_num": int}:
    with open(in_art, "r") as f:
        content = f.read()
    with open("bar.txt", "w") as f:
        f.write(content * 2)

    return {
        "out_art": Path("bar.txt"),
        "out_num": in_num * 2,
    }

After defining OPs, call the OPs in series in the context of a workflow, which will connect them together as a workflow and submit it finally.

In [ ]:
with Workflow(name="python") as wf:
    out = write_file(msg="HelloWorld!")
    duplicate(in_num=out["length"], in_art=out["out_art"])